# 问题一求解代码

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 数据读取
data = pd.read_excel('附件1近5年402家供应商的相关数据.xlsx', sheet_name='评价矩阵')
data = data[['供应商ID', '履约率', '稳定性系数', '需求供应比', '平均供货量', '最大供货能力', '供货总量', '供货频率', '超额供货率', '短缺供货率', '平均偏差值']] 

data = data[['履约率', '稳定性系数', '需求供应比', '平均供货量', '最大供货能力', '供货总量', '供货频率', '超额供货率', '短缺供货率','平均偏差值']]
data.describe()

,履约率,稳定性系数,需求供应比,平均供货量,最大供货能力,供货总量,供货频率,超额供货率,短缺供货率,平均偏差值
count,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000
mean,0.599081,0.566480,0.539881,59.963782,621.870647,10945.288557,0.267247,0.341902,0.468235,0.637795
std,0.289539,1.052355,0.426995,232.299231,3322.264018,41336.382900,0.300734,0.261590,0.284865,0.299131
min,0.030303,0.064550,0.000718,1.000000,1.000000,28.000000,0.004167,0.000000,0.000000,0.007399
25%,0.342564,0.209836,0.109018,2.142857,6.000000,44.250000,0.071875,0.127213,0.180804,0.389966
50%,0.579587,0.271683,0.492312,2.732051,9.000000,75.000000,0.116667,0.250000,0.487146,0.676644
75%,0.894323,0.436283,0.961948,6.072581,30.000000,341.500000,0.375000,0.569196,0.723001,0.934344
max,1.000000,8.259747,1.367589,2928.178571,36972.000000,354887.000000,1.000000,0.908333,1.000000,0.999505


正向化处理

In [3]:
def normalize_indicators(data_matrix, indicator_types, ideal_values=None):
    """
    根据指标类型对数据进行正向化处理
    
    :param data_matrix: 二维数组，行为供应商，列为指标
    :param indicator_types: 列表，指示每个指标的类型
        'positive' - 极大型指标（越大越好）
        'negative' - 极小型指标（越小越好）
        'moderate' - 中间型指标（越接近某个值越好）
    :param ideal_values: 中间型指标的理想值列表
    :return: 正向化后的数据矩阵
    """
    if isinstance(data_matrix, pd.DataFrame):
        data_matrix = data_matrix.values
    normalized_matrix = np.zeros_like(data_matrix, dtype=float)
    n_indicators = data_matrix.shape[1]
    
    for j in range(n_indicators):
        col = data_matrix[:, j]
        indicator_type = indicator_types[j]
        
        if indicator_type == 'positive':
            # 极大型指标：无需处理
            normalized_matrix[:, j] = col
            
        elif indicator_type == 'negative':
            # 极小型指标：转化为极大型
            # 方法1：取倒数（适用于无零值的情况）
            # normalized_matrix[:, j] = 1 / (col + 1e-10)
            
            # 方法2：用最大值减去当前值（更常用）
            max_val = np.max(col)
            normalized_matrix[:, j] = max_val - col
            
        elif indicator_type == 'moderate':
            # 中间型指标：转化为极大型（越接近理想值越好）
            if ideal_values is None:
                raise ValueError("必须为中间型指标提供理想值")
                
            ideal = ideal_values[j]
            # 计算与理想值的绝对偏差
            deviation = np.abs(col - ideal)
            # 最大偏差（避免除零）
            max_dev = np.max(deviation) if np.max(deviation) > 0 else 1
            # 转化为极大型：偏差越小越好
            normalized_matrix[:, j] = 1 - deviation / max_dev
    
    return normalized_matrix

In [4]:
indicator_types = ['positive', 'positive', 'moderate', 'positive', 'positive', 'positive', 'positive', 'negative' , 'negative', 'negative']
ideal_values = [None, None, 1, None, None, None, None, None, None, None]
normalize_matrix = normalize_indicators(data, indicator_types, ideal_values)

normalize_matrix


array([[0.27472527, 0.28008449, 0.21155523, ..., 0.79844322, 0.20879121,
        0.12504661],
       [0.74736842, 0.22812198, 0.88341145, ..., 0.59254386, 0.67368421,
        0.50112361],
       [0.95979899, 0.71737208, 0.92003504, ..., 0.15456449, 0.85427136,
        0.84046074],
       ...,
       [0.18055556, 0.14924161, 0.0650857 , ..., 0.78333333, 0.16666667,
        0.0205973 ],
       [0.43283582, 0.28817208, 0.33452229, ..., 0.71430348, 0.37313433,
        0.20450549],
       [0.10958904, 0.10823383, 0.03584234, ..., 0.88093607, 0.05479452,
        0.03186457]], shape=(402, 10))

topsis+熵权法

In [5]:
def topsis_method(data_matrix):
    """
    TOPSIS方法完整实现
    :param data_matrix: 二维数组，行为供应商，列为指标
    :return: 
        weights: 指标权重数组
        scores: 供应商TOPSIS得分数组
        rankings: 供应商排名数组
    """
    # 1. 矩阵归一化（向量归一化）
    # 计算每列的平方和
    col_sums = np.sqrt(np.sum(data_matrix**2, axis=0))
    # 避免除零错误
    col_sums[col_sums == 0] = 1e-10
    # 归一化矩阵
    norm_matrix = data_matrix / col_sums
    
    # 2. 利用熵权法确定指标权重
    # 计算指标比重
    p_matrix = norm_matrix / np.sum(norm_matrix, axis=0)
    
    # 计算信息熵
    m = data_matrix.shape[0]  # 供应商数量
    k = 1 / np.log(m)  # 熵计算系数
    e_j = np.zeros(data_matrix.shape[1])
    
    for j in range(data_matrix.shape[1]):
        col = p_matrix[:, j]
        # 避免log(0)错误
        with np.errstate(divide='ignore', invalid='ignore'):
            entropy = np.sum(col * np.log(col + 1e-10))
        e_j[j] = -k * entropy
    
    # 计算信息效用值
    d_j = 1 - e_j
    
    # 计算权重
    weights = d_j / np.sum(d_j)
    
    # 3. 构建加权规范化矩阵
    weighted_matrix = norm_matrix * weights
    
    # 4. 确定正负理想解
    positive_ideal = np.max(weighted_matrix, axis=0)
    negative_ideal = np.min(weighted_matrix, axis=0)
    
    # 5. 计算欧氏距离
    d_positive = np.sqrt(np.sum((weighted_matrix - positive_ideal)**2, axis=1))
    d_negative = np.sqrt(np.sum((weighted_matrix - negative_ideal)**2, axis=1))
    
    # 6. 计算相对接近度（TOPSIS得分）
    scores = d_negative / (d_positive + d_negative + 1e-10)
    
    # 7. 供应商排名
    rankings = np.argsort(-scores)  # 从高到低排序
    rankings = rankings + 1
    
    return weights, scores, rankings

In [6]:
weights, scores, rankings = topsis_method(normalize_matrix)

print(weights)
# print(sum(weights))
print(scores)
print(rankings)


[0.01283731 0.07476976 0.03346043 0.22277316 0.29124713 0.24405432
 0.05220211 0.01366186 0.01579568 0.03919825]
[0.00656866 0.01688835 0.03714947 0.00658242 0.02982813 0.00459981
 0.03601759 0.00862041 0.00535952 0.0080389  0.0174377  0.00464327
 0.01630794 0.01409065 0.00484421 0.01809404 0.01242557 0.01103132
 0.00780715 0.00961771 0.0090232  0.00860953 0.01723283 0.01141687
 0.02349272 0.01446256 0.01656489 0.01222874 0.00617699 0.01670258
 0.09483946 0.01340055 0.01800497 0.00467383 0.01455049 0.01691542
 0.13509726 0.00827815 0.01095563 0.06179411 0.00829601 0.00952569
 0.00534299 0.01600657 0.00628519 0.02538805 0.00591348 0.01229545
 0.01205234 0.01534073 0.00474681 0.01449319 0.01726616 0.02030728
 0.05237086 0.01022423 0.00802527 0.00553267 0.00928043 0.01697714
 0.00620422 0.01722034 0.00547807 0.01838572 0.01797127 0.01997
 0.02900047 0.00512251 0.01308831 0.0102213  0.0120416  0.00700912
 0.01648866 0.03637283 0.02243023 0.030973   0.01006062 0.03036696
 0.00671507 0.04723

In [7]:
print(f'前50个供应商排名：{rankings[:50]}')
print(f'前50个供应商得分：{scores[rankings[:50] - 1]}')


前50个供应商排名：[201 140 348 151 229 361 374 108 139 126 330 395 308 340 282 275 329 307
 268 131 356 306 194  37 352 143 247 266  31 284 294 346 365 338  40 364
 367  55 244  80 123 218  86 210   3 114  74   7 273 189]
前50个供应商得分：[0.69017221 0.57874042 0.54050957 0.46643124 0.43405054 0.40975943
 0.38494464 0.3616791  0.29389511 0.29135439 0.26827551 0.26106294
 0.25025321 0.241503   0.23691883 0.22899974 0.22592303 0.21687017
 0.20273352 0.1992947  0.18908362 0.18695957 0.16920505 0.13509726
 0.13459975 0.13282607 0.12286099 0.10416237 0.09483946 0.08563344
 0.08436761 0.0835308  0.07685715 0.06683836 0.06179411 0.05677366
 0.0557646  0.05237086 0.047256   0.04723789 0.04720454 0.04618287
 0.04513091 0.04142406 0.03714947 0.03711223 0.03637283 0.03601759
 0.03377182 0.03355001]


In [8]:
#? 美化输出
ranking_df = pd.DataFrame({
    '供应商排名': rankings[:],
    '得分': scores[rankings[:] - 1]
})

# 设置索引从1开始
ranking_df.index = range(1, 402 + 1)

# 设置列名
ranking_df.columns = ['供应商编号', 'TOPSIS得分']

# 设置显示格式
pd.set_option('display.float_format', lambda x: '{:.6f}'.format(x))

# 打印美化后的结果
print("前50名供应商排名及得分情况：")
print("=" * 50)
print(ranking_df)
print("=" * 50)

前50名供应商排名及得分情况：
     供应商编号  TOPSIS得分
1      201  0.690172
2      140  0.578740
3      348  0.540510
4      151  0.466431
5      229  0.434051
..     ...       ...
398    402  0.004366
399    400  0.004346
400    372  0.004297
401    251  0.004169
402    355  0.004148

[402 rows x 2 columns]


In [9]:
try:   
    !jupyter nbconvert --to python q1.ipynb
    # python即转化为.py，script即转化为.html
except:
    pass

'jupyter' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


In [ ]:
# 导出为图表
data.head()


,履约率,稳定性系数,需求供应比,平均供货量,最大供货能力,供货总量,供货频率,超额供货率,短缺供货率,平均偏差值
0,0.274725,0.280084,0.212121,1.960000,6,49,0.104167,0.109890,0.791209,0.874459
1,0.747368,0.228122,0.883495,3.845070,67,273,0.295833,0.315789,0.326316,0.498382
2,0.959799,0.717372,0.920092,68.785340,387,13138,0.795833,0.753769,0.145729,0.159045
3,0.320388,0.309929,0.089762,1.939394,8,64,0.137500,0.097087,0.796117,0.938289
4,0.938596,0.794324,1.057204,64.598131,128,6912,0.445833,0.798246,0.096491,0.073417
